# Imports

In [ ]:
from Models.PPO.PPO_Agent import PPO_Agent
from Models.DDQN.DDQN_Agent import DDQN_Agent
from Models.DDQN.PRB import PrioritizedReplayBuffer
from stable_baselines3 import A2C
import slimevolleygym.mlp as mlp
from slimevolleygym.mlp import Model
import torch
import slimevolleygym
from slimevolleygym import BaselinePolicy
from utils import convert_to_vector, convert_to_value
import types
import json

# Select Device

In [ ]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')

# Print the device as a check
print("Device used: ", DEVICE)

# Load all the models as a list

In [ ]:
env = slimevolleygym.SlimeVolleyEnv()
models = []

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# PPO Baseline
agent = PPO_Agent(12, 6, DEVICE, mlp_layers=[64, 64])
agent.load_models("Logging/PPO-BASELINE/20240411-150526-lr-0.0003-entcoef-0.1-mlp-64-kl-0.03", 1, 18492436)

# PPO Selfplay
agent1 = PPO_Agent(12, 6, DEVICE, mlp_layers=[64, 64])
agent1.load_models("Logging/PPO-SELFPLAY/20240410-171658-lr-0.0003-entcoef-0", 1, 18534177)

# Genetic agent
agent1 = Model(mlp.games['slimevolleylite'])
with open('Logging/GENETIC-SELFPLAY/20240409-021844-numagents-128-totalnumgames-546000/game_546000') as f:
    d = json.load(f)
    agent1.set_model_params(d[0])
def select_action(self, state, greedy=False):
    action = self.predict(state, mean_mode=greedy)
    action = (action > 0).astype(int) # Anything positive means a 1, 0 or negative means a 0
    return convert_to_value(action), None
def evaluation_mode(self):
    pass
agent1.select_action = types.MethodType(select_action, agent1)
agent1.evaluation_mode = types.MethodType(evaluation_mode, agent1)

# A2C Baseline
model = A2C.load("Logging/A2C-BASELINE-LIBRARY/20240414-154203-lr-0.0003-entcoef-0.1-mlp-64", print_system_info=True)

# DDQN Baseline
# buffer = PrioritizedReplayBuffer(
#         buffer_size = 1, 
#         state_dim = 1, 
#         alpha = 1, 
#         beta_init = 1, 
#         device = DEVICE
#     )
# agent1 = DDQN_Agent(state_dim = 12,
#                     action_dim = 6,
#                     hidden_layer_shape = 256,
#                     device = DEVICE,
#                     lr = 0,
#                     gamma = 0,
#                     batch_size = 0,
#                     epsilon = 0)
# agent1.load("Logging/DDQN-BASELINE/20240414-052734-lr-0.0005", 1, 10589200, buffer)
# agent1.select_action_original = agent1.select_action
# def select_action(self, state, greedy=False, deterministic=False):
#     action = self.select_action_original(state, deterministic=greedy)
#     return action, None
# agent1.select_action = types.MethodType(select_action, agent1)

# DDQN Self-play
# buffer = PrioritizedReplayBuffer(
#         buffer_size = 1, 
#         state_dim = 1, 
#         alpha = 1, 
#         beta_init = 1, 
#         device = DEVICE
#     )
# agent1 = DDQN_Agent(state_dim = 12,
#                     action_dim = 6,
#                     hidden_layer_shape = 256,
#                     device = DEVICE,
#                     lr = 0,
#                     gamma = 0,
#                     batch_size = 0,
#                     epsilon = 0)
# agent1.load("Logging/DDQN-BASELINE/20240414-052734-lr-0.0005", 1, 10589200, buffer)
# agent1.select_action_original = agent1.select_action
# def select_action(self, state, greedy=False, deterministic=False):
#     action = self.select_action_original(state, deterministic=greedy)
#     return action, None
# agent1.select_action = types.MethodType(select_action, agent1)

# Baseline
agent2 = BaselinePolicy()
def select_action(self, state, greedy=False):
    action = self.predict(state)
    return convert_to_value(action), None
def evaluation_mode(self):
    pass
agent2.select_action = types.MethodType(select_action, agent2)
agent2.evaluation_mode = types.MethodType(evaluation_mode, agent2)

# Random agent
agent2 = BaselinePolicy()
def select_action(self, state, greedy=False):
    action = convert_to_value(env.action_space.sample())
    return action, None
def evaluation_mode(self):
    pass
agent2.select_action = types.MethodType(select_action, agent2)
agent2.evaluation_mode = types.MethodType(evaluation_mode, agent2)
